In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import json
import re

In [2]:
from utils import tf_lower_and_split_punct,tf_lower_and_split_punct_1

In [3]:
df = pd.read_csv('data/english_hindi.csv',index_col=0)

In [4]:
print(len(df))
df.head()

64131


,English,Hindi
0,To give adequate medical cover and to carry ou...,यहीं के पशुओं के स्वास्थ्य-सुधार तथा पशु-पालन ...
1,and she is not dazzled by anything.”,और किसी से प्रभावित नहीं होती।”
2,The Ter temple now containing a Trivikrama ima...,तेर के मंदिर में अब तक एक त्रिविक्रम की मूर्ति...
3,Which is(Lodurva Jain Temple)attract the big n...,के लिए जाना जाता है जो वर्ष भर तीर्थयात्राओं क...
4,monotheism,अद्वैतवाद


In [5]:
df[df.isnull().any(axis = 1)]

,English,Hindi


In [6]:
df.head()

,English,Hindi
0,To give adequate medical cover and to carry ou...,यहीं के पशुओं के स्वास्थ्य-सुधार तथा पशु-पालन ...
1,and she is not dazzled by anything.”,और किसी से प्रभावित नहीं होती।”
2,The Ter temple now containing a Trivikrama ima...,तेर के मंदिर में अब तक एक त्रिविक्रम की मूर्ति...
3,Which is(Lodurva Jain Temple)attract the big n...,के लिए जाना जाता है जो वर्ष भर तीर्थयात्राओं क...
4,monotheism,अद्वैतवाद


In [7]:
# df.drop(df.index[indexes],axis = 0,inplace = True)
indexes = df[df['Hindi'].apply(lambda x:True if (len(x)<=2) or (re.compile('^[\w\s\.,0-9]+$',re.ASCII).findall(x)) else False)].index
df.drop(df.index[indexes],axis =0,inplace = True)


In [8]:
df.dropna(inplace = True)
print(df.isnull().any())

English    False
Hindi      False
dtype: bool


my_list = list(df['Hindi'].apply(lambda x: re.findall('[a-zA-HJ-Z]+',x.strip(',').strip('!').strip('.').strip('?').strip('I'))))
print('rows that contain english alphabets in Hindi: ',len([x for x in my_list if len(x)]))
indexes = []
for i,j in enumerate(my_list):
    if len(j):
        indexes.append(i)

df.drop(df.index[indexes],axis = 0,inplace = True)

my_list = list(df['Hindi'].apply(lambda x: re.findall('[a-zA-HJ-Z]+',x)))
print('rows that contain english alphabets after droping in Hindi: ',len([x for x in my_list if len(x)]))


In [9]:
text_pairs =list(zip(df['English'],df['Hindi']))
random.seed(42)
random.shuffle(text_pairs)
print(text_pairs[:5])

[('According to section 79 central parliament has two important houses other than president Rajasabha of the state and Loksabha,of the people.', 'भारत के संविधान की धारा 79 के अनुसार केन्द्रीय संसद की परिषद् में राष्ट्रपति तथा दो सदन है जिन्हें राज्यों की परिषद् राज्यसभा तथा लोगों का सदन लोकसभा के नाम से जाना जाता है।'), ('taluqadars', 'तालुकदार'), ('Nepal(a democratic republic) is a South Asian Himalayan Country.', 'नेपाल (आधिकारिक रूप में संघीय लोकतान्त्रिक गणराज्य नेपाल) ( ) एक दक्षिण एशियाई भूपरिवेष्ठित हिमालयी राष्ट्र है।'), ('witnessing ahswathama', 'साक्षी अश्वत्थामा'), ('solved everything in 20 minutes.', 'सारे सवाल 20 मिनट में हल कर दिए.')]


In [10]:
num_val_samples = int(0.1 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:]
# test_pairs = text_pairs[num_train_samples+num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
# print(f'{len(test_pairs)} test pairs')

63999 total pairs
57600 training pairs
6399 validation pairs


#### approx vocab size

In [11]:
list1 = list(df['English'])
list1 = [word.strip().strip('!').strip('.').strip(',').strip('?').lower() for sentence in list1 for word in sentence.split(' ') if len(word)]

print('approx english vocab:',len(set(list1)))

list2 = list(df['Hindi'])
list2 = [word.strip('!').strip('.').strip(',').strip('?') for sentence in list2 for word in sentence.split(' ')]
print('approx hindi vocab:',len(set(list2)))

approx english vocab: 55726
approx hindi vocab: 62726


In [12]:
list1 = list(set(list1))
print(len(list1))

55726


In [13]:
for i in range(10,20):
    print(list1[i*10:i*10+10])

['bharatam-', 'whales', 'published:istory', 'farce', 'micro-decisions', '“”freedom“”', 'mahboobnagar', 'basanta', 'occult', 'tilling']
['cortisol', 'currently', 'despicable', '2006)', 'dropadi', 'shatrughan', '2912kms', 'pro-poor', 'profitability', "wassalam's"]
['bi-monthly', 'isolation', 'wicket-keeper', 'personalized', 'pawar', 'language,caste', 'chapati', 'viruses', 'disband', 'non-muslim']
['bowl', 'freemasons', 'miles-long', 'rosogulla', 'vir', 'evenly', 'parliamentarian', 'saunaka', 'dizygotic', 'pecked']
['ingeniousness', 'polls', 'narrating', 'panthas', 'permanent', 'revering', 'espo', 'yaminaddaula', 'foursome.”', 'consul']
['opportunities', 'bonds-which', '1.09.1939', 'tashkent1', 'comfortable', '“”the', '74%', 'blistering', '174', 'akshay']
['projection', 'lohagarh', 'kern', 'bindings', 'beheaded', 'brain', 'aranyakas', '54.”)', 'myth', 'effect.increased']
['shelved', 'allotments', 'group:', 'enjoyable', 'limpyataran', 'bishop', 'huri', 'pronouncing', 'newborn', 'changez']


In [14]:
vocab_size = 15000
sequence_length = 50
batch_size = 64

In [15]:
# vectorization
eng_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
    standardize=tf_lower_and_split_punct
)

hin_vectorization = tf.keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length+1,
    standardize=tf_lower_and_split_punct_1
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_hin_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
hin_vectorization.adapt(train_hin_texts)

In [16]:
#save the vectorization layers

eng_vocab = eng_vectorization.get_vocabulary()
    
with open('text_vectorization_files/english_vocab_for_eng_hin.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    
hin_vocab = hin_vectorization.get_vocabulary()
    
with open('text_vectorization_files/hindi_vocab_for_eng_hin.json', 'w', encoding='utf-8') as f:
    json.dump(hin_vocab, f)
    

In [17]:
print(len(eng_vocab))
print(len(hin_vocab))
print(eng_vocab[:20])
print(hin_vocab[:20])

15000
15000
['', '[UNK]', 'the', '[SOS]', '[EOS]', ',', '.', 'of', 'and', 'in', 'to', 'a', 'is', 'that', 'for', 'was', 'it', 'as', 'by', 'on']
['', '[UNK]', '[SOS]', '[EOS]', 'के', 'में', 'है', 'की', ',', 'और', 'से', 'का', 'को', 'हैं', 'पर', 'एक', 'कि', 'भी', 'यह', 'लिए']


In [18]:
list1 = [x for x in eng_vocab if len(x)==10]
len(list1)

4614

In [19]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    hin = hin_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": hin[:, :-1],
        },
        hin[:, 1:],
    )
    
def make_dataset(pairs):
    eng_texts, hin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    hin_texts = list(hin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, hin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
# test_ds = make_dataset(test_pairs)

In [20]:
for inp,tar in train_ds.take(1):
    print(inp['encoder_inputs'][:2])
    print(inp['decoder_inputs'][:2])
    print(tar[:2])

tf.Tensor(
[[    3     8    98    42    55  3634  4527  2940    84     8  4001  4514
      9   974     8  1376   443     8   194     9  1175    22     2   494
      8     2   774     6     4     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [    3     9 13299     5   298    89   142  1435     6     4     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]], shape=(2, 50), dtype=int64)
tf.Tensor(
[[    2  1517     9  1776     5   106     7     1     1     1     9    95
    664     9 11476     4    41  6747    27 11168  3912     3     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [    2     1  

In [21]:
vocab_size = 15000
units_1 = 128

In [22]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)

        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim

        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim = embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(dense_dim,activation = 'relu'),
                tf.keras.layers.Dense(embed_dim)
            ]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def call(self,inputs,mask = None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,None,:],dtype = tf.int32)
        else:
            padding_mask = None
        attention_output = self.mha(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask
        )
        proj_input = self.layernorm_1(attention_output+inputs)
        proj_output = self.dense_proj(proj_input)
        output = self.layernorm_2(proj_input + proj_output)
        return output
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim':self.embed_dim,
            'dense_dim':self.dense_dim,
            'num_heads': self.num_heads,
        })
        return config

In [23]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self,sequence_length,vocab_size,embed_dim,**kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        
        self.token_embeddings = tf.keras.layers.Embedding(
            input_dim = vocab_size,
            output_dim  = embed_dim
        )

        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim = sequence_length,
            output_dim = embed_dim
        )

    def call(self,inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0,limit = length,delta =1)
        embeded_tokens = self.token_embeddings(inputs)
        embeded_position = self.position_embeddings(positions)
        return embeded_tokens + embeded_position
    
    def compute_mask(self,inputs,mask =None):
        if mask is not None:
            return tf.not_equal(inputs,0)
        else:
            return None
    def get_config(self):
        config = super().get_config()
        config.update({
            'vocab_size': self.vocab_size,
            'sequence_length': self.sequence_length,
            'embed_dim': self.embed_dim,
        })
        return config

In [24]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self,embed_dim,latent_dim,num_heads,**kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,key_dim=embed_dim)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads = num_heads,key_dim=embed_dim)

        self.dense_proj = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(latent_dim,activation = 'relu'),
                tf.keras.layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def call(self,inputs,encoder_outputs,mask=None):
        casual_mask = self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:,None,:],dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask,casual_mask)
        else:
            padding_mask = None

        attention_output1 = self.mha1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask
        )

        out_1 = self.layernorm_1(inputs + attention_output1)

        attention_output2 = self.mha2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )

        out_2 = self.layernorm_2(out_1 + attention_output2)
        proj_output = self.dense_proj(out_2)
        output = self.layernorm_3(proj_output + out_2)

        return output
    
    def get_casual_attention_mask(self,inputs):
        input_shape = tf.shape(inputs)
        batch_size,sequence_length = input_shape[0],input_shape[1]
        i = tf.range(sequence_length)[:,None]
        j = tf.range(sequence_length)
        mask = tf.cast(i>=j,tf.int32)
        mask = tf.reshape(mask,(1,input_shape[1],input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size,-1),
                tf.convert_to_tensor([1,1]),
            ],
            axis =0,
        )
        return tf.tile(mask,mult)
    

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'latent_dim': self.latent_dim,
            'num_heads': self.num_heads
        })
        return config


In [25]:
tf.range(0,20,delta = 1)

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])>

In [30]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 100
latent_dim = 256
num_heads = 2

#Encoder
encoder_inputs = tf.keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = tf.keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = tf.keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = tf.keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = tf.keras.layers.Dropout(0.5)(x)

decoder_outputs = tf.keras.layers.Dense(vocab_size, activation = tf.nn.log_softmax)(x)

decoder = tf.keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


In [31]:
logits =transformer((inp['encoder_inputs'],inp['decoder_inputs']))

In [32]:
epochs = 1
losses = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
transformer.summary()

transformer.compile(
    "adam", loss = losses, metrics = ["accuracy"]
)
# transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)


Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 100) │  1,505,000 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 100) │    132,656 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, None,      │  3,233,556 │ decoder_inputs[0… │
│ (Functional)        │ 15000)            │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,871,212 (18.58 MB)

 Trainable params: 4,871,212 (18.58 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
transformer.load_weights('model_weights/english_hindi_model.weights.h5')

In [34]:
my_df = pd.read_csv('data/english_hindi_10_letter.csv',index_col=0)

In [35]:
my_df.head()

,English,Hindi
0,unlettered voluminous,अपढ़ मोटा
1,bheemghoda watermills,भीमघोड़ा पनचक्कियों
2,kalyanathe,कल्याणथे
3,patronized unaccepted,को संरक्षण अस्वीकृत
4,gothakhoor,गोथाखूर


In [36]:
test_pairs = list(zip(my_df['English'],my_df['Hindi']))
test_pairs[:5]

[('unlettered voluminous', 'अपढ़ मोटा'),
 ('bheemghoda watermills', 'भीमघोड़ा पनचक्कियों'),
 ('kalyanathe', 'कल्याणथे'),
 ('patronized unaccepted', 'को संरक्षण अस्वीकृत'),
 ('gothakhoor', 'गोथाखूर')]

In [37]:
hin_vocab = hin_vectorization.get_vocabulary()
hin_index_lookup = dict(zip(range(len(hin_vocab)), hin_vocab))
max_decoded_sentence_length = sequence_length

def decode_sentence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[SOS]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = hin_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = hin_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[EOS]":
            break
    return decoded_sentence

my_list1 = []
for _ in range(5):
    input_sentence,output_sentence = random.choice(test_pairs)
    my_list1.append(input_sentence)
    input_sentence = input_sentence.lower()
    translated = decode_sentence(input_sentence)
    print(f"input: {input_sentence}")
    print(f'correct_translation: {output_sentence}')
    print(f"translated: {translated}")
    print()

input: pollinated maashelkar
correct_translation: परागण माशेलकर
translated: [SOS] परागण माशेलकर [EOS]

input: viewpoints
correct_translation: दृष्टिकोण
translated: [SOS] दृष्टिकोण [EOS]

input: resourcess
correct_translation: संसाधन
translated: [SOS] संसाधन [EOS]

input: targetting
correct_translation: लक्ष्यीकरण
translated: [SOS] लक्ष्यीकरण [EOS]

input: pratapgarh allahabaad
correct_translation: प्रतापगढ़ इलाहाबाद
translated: [SOS] प्रतापगढ़ इलाहाबाद [EOS]



In [38]:
#some elembents
my_list = ['seamlessly', 'collection', 'deflection', 'remissions', 'articulate', 'terrrorist',
            'catwalking', 'celebrated', 'bharatendu', 'mujahideen', 'mobilizing', 'interrupts', 
            'transpires', 'classmates', 'jahawarlal', 'evangelist', 'amusements', 'habituated', 
            'ascendancy', 'admiringly', 'currencies', 'enactments', 'contracted', 'switchover vaisheshik', 
            'assistance aparbrahma', 'undetected applicants', 'mistresses suchindram', 'recuperate prosecuted', 
            'deforested glorifying', 'inoculated pleasantly', 'thereafter revengeful humiliated', 'vibheeshan parushuram kurukhatra',
            'dharmshala pakisthaan swapurusha','portuguese electicity shrotyagya',]

In [41]:
x=10
for i in my_list[x:x+10]:
    print(decode_sentence(i))

[SOS] जुटाने [EOS]
[SOS] बीच में दो [EOS]
[SOS] घटित होता है [EOS]
[SOS] सहपाठियों [EOS]
[SOS] जवाहरलाल [EOS]
[SOS] इंजीलवादी [EOS]
[SOS] एम्यूजमेंट [EOS]
[SOS] आदी [EOS]
[SOS] प्रभुत्व [EOS]
[SOS] प्रशंसापूर्वक [EOS]
